<a href="https://colab.research.google.com/github/thistle0420/twitch-livechat/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [147]:
# 動画URL
URL = 'https://www.twitch.tv/videos/1893096432'
# 動画ID
VIDEOID = URL.split('/')[-1]
# GoogleSpreadsheetのID
KEY = '1lPUR4L97naiUZcZAd2P3ebTjADqBHP_w0a5WFT1IpNw'


In [150]:
import requests
from string import Template

headers = {
    'Client-Id': 'kimne78kx3ncx6brgo4mv6wki5h1ko',
}

num = 0
output = [['オフセット', 'ユーザ名', 'コメント']]

t = Template('[{"operationName":"VideoCommentsByOffsetOrCursor","variables":{"videoID":"${videoid}","contentOffsetSeconds":${contentOffsetSeconds}},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"b70a3591ff0f4e0313d126c6a1502d79a1c02baebb288227c582044aa76adf6a"}}}]')

while True:
  print(num)
  response = requests.post('https://gql.twitch.tv/gql', headers=headers, data=t.substitute(videoid=VIDEOID, contentOffsetSeconds=num))
  if 'errors' in response.json()[0]:
    break
  edges = response.json()[0]['data']['video']['comments']['edges']

  for edge in edges:
    node = edge['node']
    contentOffsetSeconds = node['contentOffsetSeconds']
    if contentOffsetSeconds < num:
      continue

    commenter = node['commenter']
    if commenter is None:
      name = ''
    else:
      name = node['commenter']['displayName']

    fragments = node['message']['fragments']
    text = ''
    for fragment in fragments:
      if fragment['emote'] == None:
        text += fragment['text']
      else:
        text += '[' + fragment['text'] + ']'

    output.append([contentOffsetSeconds, name, text])
  else:
    num = contentOffsetSeconds + 1


0
26
32
37
41
47
54
69
85
107
164
243
290
321
358
419
470
495
510
532
581
636
658
667
714
753
825
844
869
928
939
1032
1092
1117
1171
1281
1415
1422
1426
1433
1459
1483
1589
1744
1788
1906
1956
1990
1997
2041
2124
2151
2162
2191
2248
2297
2418
2510
2556
2634
2741
2754
2882
2909
2969
2991
3095
3170
3229
3241
3248
3279
3290
3345
3387
3429
3493
3565
3660
3760
3797
3801


In [151]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

sh = gc.open_by_key(KEY)
try:
  sh.add_worksheet(VIDEOID, rows=30, cols=20)
except Exception as e :
  sh.worksheet(VIDEOID).clear()

sh.values_append(VIDEOID, {"valueInputOption": "USER_ENTERED"}, {"values": output})

{'spreadsheetId': '1lPUR4L97naiUZcZAd2P3ebTjADqBHP_w0a5WFT1IpNw',
 'updates': {'spreadsheetId': '1lPUR4L97naiUZcZAd2P3ebTjADqBHP_w0a5WFT1IpNw',
  'updatedRange': "'1893096432'!A1:C3444",
  'updatedRows': 3444,
  'updatedColumns': 3,
  'updatedCells': 10332}}